<a href="https://colab.research.google.com/github/MahmoudEssam707/Data-Tools/blob/main/Reuters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd

In [ ]:
def wrangle(reuters_path):
    # Reading Data
    reuters_data =pd.read_csv(reuters_path)
    return reuters_data

In [ ]:
# Load reuters dataset
file_path= '/content/reuters_data.csv'
reuters_data = wrangle(reuters_path=file_path)
reuters_data

,REUTERS TOPICS,TOPICS,TITLE,BODY
0,YES,cocoa,BAHIA COCOA REVIEW,Showers continued throughout the week in the B...
1,YES,grain,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE,The U.S. Agriculture Department reported the f...
2,YES,veg-oil,ARGENTINE 1986/87 GRAIN/OILSEED REGISTRATIONS,Argentine grain board figures show crop regist...
3,YES,none,USX <X> DEBT DOWGRADED BY MOODY'S,Moody's Investors Service Inc said it lowered ...
4,YES,earn,CHAMPION PRODUCTS <CH> APPROVES STOCK SPLIT,Champion Products Inc said its board of direct...
...,...,...,...,...
13146,YES,earn,EASTERN UTILITIES ASSOCIATES 3RD QTR NET,Shr 86 cts vs 74 cts Net 11.1 mln vs 8.6 m...
13147,YES,trade,"EC, U.S. PLAN HIGH-LEVEL TRADE TALKS",The European Community is willing to offer lim...
13148,YES,crude,BRITAIN BACKS U.S. STRIKE ON IRAN OIL PLATFORM,British Foreign Secretary Sir Geoffrey Howe ba...
13149,YES,acq,SIMON AND SCHUSTER TO ACQUIRE WOODHEAD-FAULKNE...,SIMON AND SCHUSTER TO ACQUIRE WOODHEAD-FAULKNE...


In [ ]:
reuters_data["BODY"][2]